# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [28]:
df_1 = data.drop(["SalesID", "ProductID"], axis=1)
df_1= df_1.groupby(by = ["CustomerID", "ProductName"]).sum(["Quantity"]).reset_index()

df_1.head(5)


,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [29]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63628 entries, 0 to 63627
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerID   63628 non-null  int64 
 1   ProductName  63628 non-null  object
 2   Quantity     63628 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [30]:
df_2 = pd.pivot_table(df_1,
                   values= "Quantity",
                   index= "ProductName",
                   columns= "CustomerID",
                   aggfunc= 'sum',
                   ).fillna(0).astype(int)
#Incluyo astype para que no haya decimales

In [31]:
df_2 

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [32]:
#Trasnsposición: las filas se convierten en columnas y las columnas en filas.
df_3 = df_2.T
df_3

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0,0,0,25,0,50,0,25,0,0,...,0,25,25,0,0,0,0,0,0,0
98069,0,0,0,25,0,25,0,0,0,25,...,0,0,0,0,0,0,0,0,0,0
98159,0,0,0,0,0,0,0,0,0,0,...,0,50,0,0,0,0,25,0,0,0


In [33]:
#pdist: calcula la distancia entre los pares de puntos de la matriz.
#se almacena en "dist_calculation" que contiene las distancias únicas entre los puntos.
#El resultado es una matriz de vector
dist_calculation = pdist(X=df_3, metric='euclidean')
dist_calculation



array([ 11.91637529,  10.48808848,  11.22497216, ..., 304.13812651,
       305.16389039, 303.10889132])

In [34]:
#Squareform:Se hace para convertir la matriz de distancia de vector en matriz cuadrada y poder aplicar squareform and pdist

dist_distribution = squareform(dist_calculation)
dist_distribution

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [35]:
dist_distribution.shape

(1000, 1000)

In [36]:
#se crea el dataframe con las distancias entre los clientes.
euclid_dist = pd.DataFrame(dist_distribution,
                           index=df_2.columns, 
                           columns=df_2.columns)

euclid_dist

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,220.501701,220.202180,219.136943,219.952268,219.733930,219.599636,219.538152,219.924987,219.827205,220.070443,...,283.945417,283.945417,302.076149,272.717803,278.388218,0.000000,273.861279,291.547595,306.186218,307.205143
98069,217.188858,215.728997,216.612557,217.437347,217.446545,217.425849,216.903204,217.294731,217.080630,216.751009,...,283.945417,283.945417,295.803989,283.945417,285.043856,273.861279,0.000000,287.228132,297.909382,294.745653
98159,228.628520,228.010965,228.081126,228.098663,227.997807,228.197283,228.028507,228.181945,227.868383,228.103047,...,283.945417,279.508497,300.000000,290.473751,300.000000,291.547595,287.228132,0.000000,304.138127,305.163890


In [37]:
euclid_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(X=df_3, metric='euclidean'))),
                                index=euclid_dist.columns,
                                columns=euclid_dist.columns)

euclid_dist_norm.head(5)
#Mismo que lo anterior pero con otro método.En este se hace todo a la vez.

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID

In [38]:
top5 = euclid_dist_norm[33].sort_values(ascending=False)[1:6]
top5

CustomerID
3317    0.087047
3535    0.087047
264     0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

In [39]:
top5_pd = pd.DataFrame(top5).reset_index()
top5_pd

,CustomerID,33
0,3317,0.087047
1,3535,0.087047
2,264,0.087047
3,2503,0.085983
4,3305,0.085638


In [40]:
#Este es utilizando el normalizado, hay que poner ascending True porque sino devuelve lo contrario.
top_5= euclid_dist[33].sort_values(ascending=True)[1:6]
top_5

CustomerID
3535    10.488088
264     10.488088
3317    10.488088
2503    10.630146
3305    10.677078
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [41]:
df_1.columns

Index(['CustomerID', 'ProductName', 'Quantity'], dtype='object')

In [42]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63628 entries, 0 to 63627
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerID   63628 non-null  int64 
 1   ProductName  63628 non-null  object
 2   Quantity     63628 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [43]:
df_1.head(5)

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [44]:
df_5 = df_1.loc[df_1['CustomerID'].isin(top5_pd['CustomerID'].values)]
df_5.head(5)

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


In [45]:
df_5['Quantity'].value_counts()

1    260
2     10
Name: Quantity, dtype: int64

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [46]:
df_6 = df_5.groupby(by= 'ProductName')['Quantity'].sum().sort_values(ascending=False)
df_6 = df_6.to_frame()
df_6

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


In [47]:
df_6.info()

<class 'pandas.core.frame.DataFrame'>
Index: 218 entries, Butter - Unsalted to Yogurt - French Vanilla
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Quantity  218 non-null    int64
dtypes: int64(1)
memory usage: 3.4+ KB


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [48]:
merged_df = df_1.merge(df_2, on='ProductName')
merged_df

,CustomerID,ProductName,Quantity,33,200,264,356,412,464,477,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,33,Apricots - Dried,1,1,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
1,412,Apricots - Dried,1,1,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
2,883,Apricots - Dried,1,1,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
3,2582,Apricots - Dried,1,1,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
4,2754,Apricots - Dried,1,1,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63623,92067,Bay Leaf,24,0,0,0,0,0,0,0,...,0,0,0,0,0,25,25,0,0,0
63624,97063,Bay Leaf,25,0,0,0,0,0,0,0,...,0,0,0,0,0,25,25,0,0,0
63625,97324,Bay Leaf,25,0,0,0,0,0,0,0,...,0,0,0,0,0,25,25,0,0,0
63626,97928,Bay Leaf,25,0,0,0,0,0,0,0,...,0,0,0,0,0,25,25,0,0,0


In [49]:
merged_df = merged_df[merged_df[33].isin([0])].sort_values('Quantity', ascending=False)
merged_df = merged_df.iloc[:5]
merged_df.head(5)


,CustomerID,ProductName,Quantity,33,200,264,356,412,464,477,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
48917,90069,Longos - Grilled Salmon With Bbq,92,0,0,0,0,0,0,0,...,0,0,0,0,0,0,25,0,0,0
55409,80694,Yeast Dry - Fermipan,84,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
35719,97029,Fenngreek Seed,75,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
10633,95017,Coffee - Irish Cream,75,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32696,97900,Scampi Tail,75,0,0,0,0,1,0,0,...,0,0,0,0,75,25,0,0,0,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [ ]:
''''
#4
top5 = euclid_dist_norm[33].sort_values(ascending=False)[1:6]
top5
top5_pd = pd.DataFrame(top5).reset_index()
top5_pd

#5
df_5 = df_1.loc[df_1['CustomerID'].isin(top5_pd['CustomerID'].values)]
df_5.head(5)

#df_6 = df_5.groupby(by= 'ProductName')['Quantity'].sum().sort_values(ascending=False)
df_6 = df_6.to_frame()
df_6

#7
merged_df = df_1.merge(df_2, on='ProductName')
merged_df = merged_df[merged_df[33].isin([0])].sort_values('Quantity', ascending=False)
merged_df = merged_df.iloc[:5]
merged_df.head(5)'''


In [50]:
recommendations = {}
list_customer = list(df_1["CustomerID"].unique())
list_customer

[33,
 200,
 264,
 356,
 412,
 464,
 477,
 639,
 649,
 669,
 694,
 756,
 883,
 891,
 1008,
 1034,
 1066,
 1072,
 1336,
 1428,
 1435,
 1534,
 1577,
 1594,
 1754,
 1839,
 1920,
 2187,
 2329,
 2503,
 2556,
 2566,
 2582,
 2617,
 2686,
 2754,
 2776,
 2902,
 2915,
 2939,
 3074,
 3253,
 3267,
 3305,
 3317,
 3472,
 3531,
 3535,
 3544,
 3885,
 3903,
 3909,
 4261,
 4479,
 4595,
 4644,
 4723,
 5104,
 5177,
 5183,
 5224,
 5230,
 5434,
 5678,
 5739,
 5832,
 5921,
 5968,
 5986,
 5997,
 6001,
 6185,
 6317,
 6352,
 6367,
 6447,
 6543,
 6734,
 6817,
 6851,
 7021,
 7134,
 7223,
 7543,
 7606,
 7623,
 7772,
 7827,
 7987,
 7999,
 8316,
 8367,
 8372,
 8539,
 8711,
 8917,
 8962,
 9047,
 9132,
 9260,
 9302,
 9533,
 9546,
 9597,
 9771,
 9804,
 10016,
 10094,
 10114,
 10224,
 10311,
 10418,
 10739,
 11023,
 11057,
 11083,
 11102,
 11253,
 11325,
 11380,
 11441,
 11443,
 11614,
 11628,
 11744,
 11883,
 11933,
 12059,
 12206,
 12232,
 12381,
 12529,
 12561,
 12596,
 12598,
 12852,
 12888,
 13062,
 13118,
 13338,
 

In [ ]:
recommendations = {}
list_customer = list(df_1["CustomerID"].unique())

for customer in list_customer:
    top5 = euclid_dist_norm[customer].sort_values(ascending=False)[1:6]
    top5_pd = pd.DataFrame(top5).reset_index()
    df_5 = df_1.loc[df_1['CustomerID'].isin(top5_pd['CustomerID'].values)]
    df_6 = df_5.groupby(by= 'ProductName')['Quantity'].sum().sort_values(ascending=False)
    df_6 = df_6.to_frame()
    merged_df = df_1.merge(df_2, on='ProductName')
    merged_df = merged_df[merged_df[customer].isin([0])].sort_values('Quantity', ascending=False)
    merged_df = merged_df.iloc[:5]
    recommendations[customer] = merged_df
    print(recommendations[customer])


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [ ]:
df_recommendations = recommendations.T

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [51]:
cosine_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(df_2.T, 'cosine'))),
                                index=df_2.columns,
                                columns=df_2.columns)
cosine_dist_norm

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.513809,0.521925,0.560225,0.530406,0.536686,0.542873,0.540381,0.533190,0.535224,0.526495,...,0.532674,0.539309,0.522883,0.542570,0.569539,1.000000,0.562050,0.543657,0.531363,0.519127
98069,0.528795,0.573177,0.548538,0.520138,0.519628,0.520835,0.534221,0.525789,0.531918,0.541402,...,0.529233,0.535979,0.530224,0.518742,0.554911,0.562050,1.000000,0.548011,0.542143,0.536728
98159,0.520208,0.537284,0.538340,0.536202,0.538710,0.533856,0.535601,0.535704,0.544926,0.535721,...,0.542777,0.556893,0.535487,0.521507,0.541404,0.543657,0.548011,1.000000,0.543258,0.531447


In [60]:
recommendations2 = {}
list_customer2 = list(df_1["CustomerID"].unique())

for customer2 in list_customer2:
    top5 = cosine_dist_norm[customer2].sort_values(ascending=False)[1:6]
    top5_pd = pd.DataFrame(top5).reset_index()
    df_5 = df_1.loc[df_1['CustomerID'].isin(top5_pd['CustomerID'].values)]
    df_6 = df_5.groupby(by= 'ProductName')['Quantity'].sum().sort_values(ascending=False)
    df_6 = df_6.to_frame()
    merged_df = df_1.merge(df_2, on='ProductName')
    merged_df = merged_df[merged_df[customer2].isin([0])].sort_values('Quantity', ascending=False)
    merged_df = merged_df.iloc[:5]
    recommendations2[customer2] = merged_df
    print(recommendations2[customer2])


       CustomerID                       ProductName  Quantity  33  200  264  \
48917       90069  Longos - Grilled Salmon With Bbq        92   0    0    0   
55409       80694              Yeast Dry - Fermipan        84   0    0    0   
35719       97029                    Fenngreek Seed        75   0    0    0   
10633       95017              Coffee - Irish Cream        75   0    1    0   
32696       97900                       Scampi Tail        75   0    0    0   

       356  412  464  477  ...  97697  97753  97769  97793  97900  97928  \
48917    0    0    0    0  ...      0      0      0      0      0      0   
55409    0    0    0    0  ...      0      0      0     25      0      0   
35719    0    0    1    0  ...      0      0      0      0      0      0   
10633    0    0    0    0  ...      0      0      0      0      0      0   
32696    0    1    0    0  ...      0      0      0      0     75     25   

       98069  98159  98185  98200  
48917     25      0      0      

       CustomerID                       ProductName  Quantity  33  200  264  \
48917       90069  Longos - Grilled Salmon With Bbq        92   0    0    0   
55409       80694              Yeast Dry - Fermipan        84   0    0    0   
46307       97063           Bread - French Baquette        75   0    0    0   
32696       97900                       Scampi Tail        75   0    0    0   
33097       96524        Spice - Peppercorn Melange        75   0    0    0   

       356  412  464  477  ...  97697  97753  97769  97793  97900  97928  \
48917    0    0    0    0  ...      0      0      0      0      0      0   
55409    0    0    0    0  ...      0      0      0     25      0      0   
46307    0    0    0    0  ...      0      0      0     25      0     25   
32696    0    1    0    0  ...      0      0      0      0     75     25   
33097    0    1    0    0  ...      0      0      0      0      0     25   

       98069  98159  98185  98200  
48917     25      0      0      

       CustomerID                       ProductName  Quantity  33  200  264  \
48917       90069  Longos - Grilled Salmon With Bbq        92   0    0    0   
55409       80694              Yeast Dry - Fermipan        84   0    0    0   
35719       97029                    Fenngreek Seed        75   0    0    0   
7144        96615               Veal - Eye Of Round        75   1    0    0   
32696       97900                       Scampi Tail        75   0    0    0   

       356  412  464  477  ...  97697  97753  97769  97793  97900  97928  \
48917    0    0    0    0  ...      0      0      0      0      0      0   
55409    0    0    0    0  ...      0      0      0     25      0      0   
35719    0    0    1    0  ...      0      0      0      0      0      0   
7144     0    0    0    0  ...      0      0      0      0      0      0   
32696    0    1    0    0  ...      0      0      0      0     75     25   

       98069  98159  98185  98200  
48917     25      0      0      

       CustomerID                       ProductName  Quantity  33  200  264  \
48917       90069  Longos - Grilled Salmon With Bbq        92   0    0    0   
55409       80694              Yeast Dry - Fermipan        84   0    0    0   
7144        96615               Veal - Eye Of Round        75   1    0    0   
33097       96524        Spice - Peppercorn Melange        75   0    0    0   
35719       97029                    Fenngreek Seed        75   0    0    0   

       356  412  464  477  ...  97697  97753  97769  97793  97900  97928  \
48917    0    0    0    0  ...      0      0      0      0      0      0   
55409    0    0    0    0  ...      0      0      0     25      0      0   
7144     0    0    0    0  ...      0      0      0      0      0      0   
33097    0    1    0    0  ...      0      0      0      0      0     25   
35719    0    0    1    0  ...      0      0      0      0      0      0   

       98069  98159  98185  98200  
48917     25      0      0      

       CustomerID                       ProductName  Quantity  33  200  264  \
48917       90069  Longos - Grilled Salmon With Bbq        92   0    0    0   
55409       80694              Yeast Dry - Fermipan        84   0    0    0   
7144        96615               Veal - Eye Of Round        75   1    0    0   
35719       97029                    Fenngreek Seed        75   0    0    0   
32696       97900                       Scampi Tail        75   0    0    0   

       356  412  464  477  ...  97697  97753  97769  97793  97900  97928  \
48917    0    0    0    0  ...      0      0      0      0      0      0   
55409    0    0    0    0  ...      0      0      0     25      0      0   
7144     0    0    0    0  ...      0      0      0      0      0      0   
35719    0    0    1    0  ...      0      0      0      0      0      0   
32696    0    1    0    0  ...      0      0      0      0     75     25   

       98069  98159  98185  98200  
48917     25      0      0      

       CustomerID                       ProductName  Quantity  33  200  264  \
48917       90069  Longos - Grilled Salmon With Bbq        92   0    0    0   
46307       97063           Bread - French Baquette        75   0    0    0   
32696       97900                       Scampi Tail        75   0    0    0   
35719       97029                    Fenngreek Seed        75   0    0    0   
7144        96615               Veal - Eye Of Round        75   1    0    0   

       356  412  464  477  ...  97697  97753  97769  97793  97900  97928  \
48917    0    0    0    0  ...      0      0      0      0      0      0   
46307    0    0    0    0  ...      0      0      0     25      0     25   
32696    0    1    0    0  ...      0      0      0      0     75     25   
35719    0    0    1    0  ...      0      0      0      0      0      0   
7144     0    0    0    0  ...      0      0      0      0      0      0   

       98069  98159  98185  98200  
48917     25      0      0      

       CustomerID                       ProductName  Quantity  33  200  264  \
48917       90069  Longos - Grilled Salmon With Bbq        92   0    0    0   
55409       80694              Yeast Dry - Fermipan        84   0    0    0   
7144        96615               Veal - Eye Of Round        75   1    0    0   
10633       95017              Coffee - Irish Cream        75   0    1    0   
32696       97900                       Scampi Tail        75   0    0    0   

       356  412  464  477  ...  97697  97753  97769  97793  97900  97928  \
48917    0    0    0    0  ...      0      0      0      0      0      0   
55409    0    0    0    0  ...      0      0      0     25      0      0   
7144     0    0    0    0  ...      0      0      0      0      0      0   
10633    0    0    0    0  ...      0      0      0      0      0      0   
32696    0    1    0    0  ...      0      0      0      0     75     25   

       98069  98159  98185  98200  
48917     25      0      0      

KeyboardInterrupt: 